In [1]:
import pandas as pd
import numpy as np

In [2]:
def class_counts(rows):
    counts = {} 
    for j in range(len(rows)):
        label = rows.iloc[j,-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [3]:
def gini(data):
    counts = class_counts(data)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(data))
        impurity -= prob_of_lbl**2
    return impurity
    

In [4]:
def partition(data,feature, current_value):
    true = []
    false = []
    for i in range(len(data)):
        if data[feature][i] == current_value :
            true.append(data.iloc[i])
        else:
            false.append(data.iloc[i])
    left = pd.DataFrame(true)
    left = left.reset_index(drop=True)
    right = pd.DataFrame(false)
    right = right.reset_index(drop=True)
    return left,right

In [5]:
def gain(data,feature,current_value,current_uncertainity):
    true,false = partition(data,feature,current_value)
    p = float(len(true)) / (len(true) + len(false))
    return current_uncertainity - p * gini(true) - (1 - p) * gini(false)

In [6]:
def find_best_feature(data, features, current_uncertainity):
    best_gain = 0
    best_feature = None
    value = None
    current_uncertainity = gini(data)
    for j in range(len(features)):
        all_possible_values = set(data[features[j]])
        for current_value in all_possible_values:
            g = gain(data,features[j],current_value,current_uncertainity)
            if(g > best_gain):
                best_gain = g
                best_feature = features[j]
                value = current_value
    return best_feature,value,best_gain

In [7]:
def dt(data , features , level):
    print()
    current_uncertainity = gini(data)
    true = 0
    false = 0
    for i in range(len(data)):
        if data.iloc[i,-1] == 0:
            false += 1
        else:
            true += 1
    if(len(features)==0):
        # do not split
        print('level :',level)
        print("Count of 0(False) = ",false)
        print("Count of 1(True) = ",true)
        print("Current entropy = ",current_uncertainity)
        print("Reached leaf node")
        return;
    
    if len(set(data.iloc[:,-1])) == 1:
        print('level :',level)
        print("Count of 0(False) = ",false)
        print("Count of 1(True) = ",true)
        print("Current entropy = ",current_uncertainity)
        print("Reached leaf node")
        return;
    
    print('level :',level)
    print("Count of 0(False) = ",false)
    print("Count of 1(True) = ",true)
    
    best_feature,value,gain = find_best_feature(data,features,current_uncertainity)
    
    left,right = partition(data, best_feature, value)

    features.remove(best_feature)
    
    print("Current Entropy is = ",current_uncertainity)
    print("Splitting on feature ",best_feature," with gain ratio")
    print(gain)
    
    level += 1
    
    dt(left,features,level)
    dt(right,features,level)

In [8]:
level = 0
data = np.array([[1,1,1],[0,1,1],[1,0,1],[0,0,0]])
data = pd.DataFrame(data)
data.columns = ['X1','X2','Y']
features = ['X1','X2']
dt(data, features, level)


level : 0
Count of 0(False) =  1
Count of 1(True) =  3
Current Entropy is =  0.375
Splitting on feature  X1  with gain ratio
0.125

level : 1
Count of 0(False) =  1
Count of 1(True) =  1
Current Entropy is =  0.5
Splitting on feature  X2  with gain ratio
0.5

level : 2
Count of 0(False) =  1
Count of 1(True) =  0
Current entropy =  0.0
Reached leaf node

level : 2
Count of 0(False) =  0
Count of 1(True) =  1
Current entropy =  0.0
Reached leaf node

level : 1
Count of 0(False) =  0
Count of 1(True) =  2
Current entropy =  0.0
Reached leaf node
